# Curate entity identifiers

To make data queryable by an entity identifier, one needs to ensure that identifiers comply to a chosen standard. Bionty enables this by curating data against the versionized ontologies using {meth}`~bionty.Entity.curate`.

We'll demonstrate this by first curating genes and second CellMarkers where not all values can be immediately mapped.

Let's start by importing the required modules from Bionty and Pandas.

In [ ]:
from bionty import Gene, CellMarker, lookup
import pandas as pd

## Curating genes

To illustrate it, generate a DataFrame that stores a number of gene identifiers, some of them corrupted.

In [ ]:
data = {
    "gene symbol": ["A1CF", "A1BG", "FANCD1", "corrupted"],
    "hgnc id": ["HGNC:24086", "HGNC:5", "HGNC:1101", "corrupted"],
    "ensembl_gene_id": [
        "ENSG00000148584",
        "ENSG00000121410",
        "ENSG00000188389",
        "corrupted",
    ],
}
df_orig = pd.DataFrame(data).set_index("ensembl_gene_id")

In [ ]:
df_orig

To do this, we need a reference identifier (specified as the `id` parameter when initiating the Gene class), the list can be looked up via {meth}`~bionty.lookup`.

In [ ]:
lookup.gene_id

To curate the DataFrame into queryable form, we create an index that corresponds to a default identifier (By default, we use `ensembl_gene_id`).

The default behavior is to curate the index if a column name is not provided.

In [ ]:
Gene().curate(df_orig)

You may provide a column name to curate a specific column against a reference identifier.

In [ ]:
Gene(id=lookup.gene_id.hgnc_id).curate(df_orig, column="hgnc id")

When mapping symbols, the function will automatically convert the aliases into standardized symbols. In this example, `PD-1` is converted into `PDCD1`.

In [ ]:
Gene(id=lookup.gene_id.symbol).curate(df_orig, column="gene symbol")

## Looking up unmatched/unlinked terms

Depending on how the data was collected and which terminology was used, it is not always possible to curate the values. Some of the values might have used a different standard or are simply corrupted.

This section will demonstrate how to look up unmatched terms using The CellMarker entity. First, we create an example Pandas DataFrame containing a few valid and invalid cell markers.

In [ ]:
proteins = pd.DataFrame(
    index=[
        "CD14",
        "CCR7",
        "OX40",
        "CD69",
        "CD8A",
        "CD45RA",
        "CD4",
        "CD3",
        "CD137",
        "Invalid-1",
        "Invalid-2",
        "LD",
        "Time",
    ]
)

In [ ]:
cell_marker = CellMarker()

In [ ]:
cell_marker.curate(proteins)